# Obliczenie dokładności na całych obrazach

### Wczytywanie koordynatów rzeczywistych i przewidzianych dla wszystkich obrazów

In [1]:
from validating_all_images import fddb_data

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
coords = fddb_data("c:/Dev/machine_learning_2/data/", show_images=False, verbose=False)

In [4]:
import numpy as np
coords = np.array(coords, dtype=object)

Pierwsza wartość to liczba wierszy, czyli liczba sprawdzonych obrzków.
Druga wartość to liczba kolumn. Pierwsza kolumna to nazwa pliku, druga kolumna to rzeczywiste bounding_boxes wczytane z plików, trzecia kolumna to bounding_boxes wraz z wartościami decyzji z detekcji na naszym modelu RealBoost.

In [5]:
coords.shape

(11, 3)

Pierwsza kolumna - nazwa pliku. Przyda się do późniejszego rysowania prostokątów.

In [6]:
coords[0][0]

'c:/Dev/machine_learning_2/data/2002/08/31/big/img_18008.jpg'

Druga kolumna - rzeczywiste bounding boxes. W formacie:

```
np.array([j0, k0, j0 + w - 1, k0 + w - 1])
```

Aby wyrysować taki bounding box będziemy używać:

```
p1 = (k0, j0)
p2 = (k0 + w - 1, j0 + w - 1)
cv2.rectangle(i0, p1, p2, (0, 0, 255), 1)
cv2.imshow("FDDB", i0)
```

In [7]:
coords[0][1]

[array([ 80, 160, 163, 243]),
 array([ 93, 253, 173, 333]),
 array([135, 358, 200, 423])]

Trzecia kolumna - wykryte bounding boxes. Do rysowania tych bounding boxów użyjemy funkcji `draw_bounding_boxes`.

```
i0 = draw_bounding_boxes(i0, detections, color=(54, 193, 56), thickness=1)
cv2.imshow("FDDB", i0)
cv2.waitKey(0)
```

In [8]:
coords[0][2]

[(170, 90, 77, 3.088884474057463),
 (258, 98, 77, 4.1719872791425106),
 (74, 194, 77, 3.2760263724187846),
 (255, 81, 92, 3.737369740285834),
 (246, 90, 92, 3.2804569915618407),
 (255, 90, 92, 4.295105309498881),
 (169, 67, 111, 3.5541449345575797)]

### Uśrednianie wykrytych prostokątów

#### Progowanie

Wykryte prostokąty należy najpierw sprogować, czyli przefiltrować po danym progu. Wszystkie wykryte prostokąty to takie z wartością funkcji decyzyjnej $\gt 0$. Czemu tak? Żeby wygenerować wszystkie dane i móc sterować progiem decyzyjnym już po przeanalizowaniu wszystkich obrazków.

In [9]:
def reduce_by_threshold(image_row, **kwargs):
    threshold = kwargs['threshold']
    detections = np.array(image_row[2], dtype='int')
    if detections.ndim == 1:
        return (image_row[0], image_row[1], detections)
    
    detections_reduced = detections[np.where(detections[:, 3] > threshold)]
    return (image_row[0], image_row[1], detections_reduced)

coords_filtered = np.apply_along_axis(reduce_by_threshold, 1, coords, threshold=2)

#### Uśrednianie

Uśrednianie prostokątów już po filtracji wybranym progiem.

In [10]:
from haar_students_new import non_max_suppression

def bounding_boxes_reduction(image_row):
    detections = image_row[2]
    detections_reduced = non_max_suppression(detections, treshold=0.1)
    return (image_row[0], image_row[1], detections_reduced)

coords_suppresion = np.apply_along_axis(bounding_boxes_reduction, 1, coords_filtered)
coords_suppresion

array([['c:/Dev/machine_learning_2/data/2002/08/31/big/img_18008.jpg',
        list([array([ 80, 160, 163, 243]), array([ 93, 253, 173, 333]), array([135, 358, 200, 423])]),
        list([(258, 98, 77, 4), (170, 90, 77, 3), (74, 194, 77, 3)])],
       ['c:/Dev/machine_learning_2/data/2002/08/22/big/img_249.jpg',
        list([array([ 32,  64, 170, 202])]), list([(90, 59, 111, 3)])],
       ['c:/Dev/machine_learning_2/data/2002/08/17/big/img_971.jpg',
        list([array([ 90,  82, 199, 191])]), list([(93, 120, 92, 6)])],
       ['c:/Dev/machine_learning_2/data/2002/08/04/big/img_308.jpg',
        list([array([ 31, 103, 147, 219])]),
        list([(271, 204, 64, 3), (218, 152, 77, 3)])],
       ['c:/Dev/machine_learning_2/data/2002/07/28/big/img_362.jpg',
        list([array([ 50,  86, 209, 245])]),
        list([(108, 289, 64, 3), (108, 93, 92, 3)])],
       ['c:/Dev/machine_learning_2/data/2002/08/12/big/img_142.jpg',
        list([array([ 97, 179, 170, 252])]),
        list([(180, 10

### Obliczanie dokładności na danych

Dla każdego uśrednionego prostokąta sprawdzamy czy się pokrywa z innym zgodnie z wymaganiem `iou > 0.5`. 

## Wizualizacja - zapisywanie obrazów z detekcją

In [11]:
import cv2
from haar_students_new import draw_bounding_boxes

def batch_drawing(image_row):
    file_path = image_row[0]
    ground_truth = image_row[1]
    detections = image_row[2]
    ground_truth = [[bb[1], bb[0], bb[2] - bb[0] - 1, None] for bb in ground_truth]
    
    i = cv2.imread(file_path)
    i = draw_bounding_boxes(i, detections, color=(54, 193, 56), thickness=2)
    i = draw_bounding_boxes(i, ground_truth, color=(70, 8, 210), thickness=2)
    
    cv2.imwrite(file_path+"_detection.jpg", i)
    
    print(file_path, len(ground_truth), len(detections))

np.apply_along_axis(batch_drawing, 1, coords_suppresion)

c:/Dev/machine_learning_2/data/2002/08/31/big/img_18008.jpg 3 3
c:/Dev/machine_learning_2/data/2002/08/22/big/img_249.jpg 1 1
c:/Dev/machine_learning_2/data/2002/08/17/big/img_971.jpg 1 1
c:/Dev/machine_learning_2/data/2002/08/04/big/img_308.jpg 1 2
c:/Dev/machine_learning_2/data/2002/07/28/big/img_362.jpg 1 2
c:/Dev/machine_learning_2/data/2002/08/12/big/img_142.jpg 1 2
c:/Dev/machine_learning_2/data/2002/08/26/big/img_61.jpg 1 0
c:/Dev/machine_learning_2/data/2002/08/14/big/img_422.jpg 1 1
c:/Dev/machine_learning_2/data/2002/07/19/big/img_607.jpg 2 2
c:/Dev/machine_learning_2/data/2003/01/15/big/img_717.jpg 2 2
c:/Dev/machine_learning_2/data/2002/08/01/big/img_1475.jpg 1 1


array([None, None, None, None, None, None, None, None, None, None, None],
      dtype=object)